<a href="https://colab.research.google.com/github/hyunbini/Project/blob/main/IDSL_LAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Library

In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from google.colab import drive
drive.mount('/content/drive')
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import cv2
import requests

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Function used in code

In [78]:
def check_nan(file):
  result = file.isnull().sum()
  return result

In [79]:
#Function that extracts user_id who wrote one or more reviews
#input : series(pandas dataframe)
#output : result(list)
def make_list(series1):
  result = []
  for i in range(series1.size):
    #Add user_id who have written reviews more than once to the new list
    if series1[i]>1:
      result.append(series1.index[i])
  return result

In [80]:
#Function that erases unnecessary characters and makes a list of only the necessary data
#input : list1(list)
#output : result_listset(list)
def remove_special(list1):
  #Change list to string
  for i in range(len(list1)):
    list1[i] = str(list1[i])
  listset_str = ' '.join(list1)
  #Remove Special characters to space
  listset_str = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', "",listset_str)
  #Split the data by space
  result_listset = listset_str.split(' ')
  return result_listset

In [81]:
#Function that labels data composed of complex characters numerically based on ascending order\
#input : list1(list to be replaced), list2(reference list), dic(dictionary to store the values of reference list and its unique labeling numbers)
#output : list_df(Dataframe made from a labeled list)
def labeling_user_id(list1,list2,dic):
  i = 0
  for data in list1:
    if data in list2:
      if data in dic.keys():
        list1[i] = dic[data]
      else:
        dic[data] = i
        list1[i] = i
      i = i+1
    else:
      list1[i] = i
      i = i+1
  for y in range(len(list1)):
    list1[y] = list1[y]+1
  list_df = pd.DataFrame(data=list1, columns= ['labeling_user_id'])
  return list_df

In [213]:
def make_label_list(dataset_name):
  emptylist = []
  for i in range(len(dataset_name)):
    emptylist.append(i)
    i = i+1
  list_df_1 = pd.DataFrame(data=emptylist, columns=['labeling_list'])
  return list_df_1

Import Data and Remove NaN Value and Unuseful Columns



In [83]:
meta_json = pd.read_json('/content/drive/MyDrive/IDSL Recruitment/[Data 1] Eng - Review Analysis/metadata.json')
review_json = pd.read_json('/content/drive/MyDrive/IDSL Recruitment/[Data 1] Eng - Review Analysis/reviews.json')
print(meta_json.head())
print(review_json.head())
print(check_nan(meta_json))
print(check_nan(review_json))

  main_category                                              title  \
0    All Beauty  Howard LC0008 Leather Conditioner, 8-Ounce (4-...   
1    All Beauty  Lurrose 100Pcs Full Cover Fake Toenails Artifi...   
2    All Beauty                         Gold extatic Musk EDT 90ml   
3    All Beauty  Brand New Headrang Face line Contour V-line Ma...   
4    All Beauty  7 Packs Deep Wave Crochet Hair 22 Inch Deep wa...   

   average_rating  rating_number  \
0             4.8             10   
1             3.7             35   
2             3.7              2   
3             3.3              4   
4             3.4             10   

                                            features  \
0                                                 []   
1  [The false toenails are durable with perfect l...   
2  [Extatic Balmain Gold Musk By Balmain Edt Spra...   
3                                                 []   
4                                                 []   

                         

In [84]:
metadata = meta_json.drop(['rating_number','categories','videos', 'bought_together', 'subtitle', 'author'],axis = 1)
print(metadata.head())
review = review_json.drop(['title', 'text','images','timestamp','user_id','helpful_vote','verified_purchase'],axis=1)
print(review.head())

  main_category                                              title  \
0    All Beauty  Howard LC0008 Leather Conditioner, 8-Ounce (4-...   
1    All Beauty  Lurrose 100Pcs Full Cover Fake Toenails Artifi...   
2    All Beauty                         Gold extatic Musk EDT 90ml   
3    All Beauty  Brand New Headrang Face line Contour V-line Ma...   
4    All Beauty  7 Packs Deep Wave Crochet Hair 22 Inch Deep wa...   

   average_rating                                           features  \
0             4.8                                                 []   
1             3.7  [The false toenails are durable with perfect l...   
2             3.7  [Extatic Balmain Gold Musk By Balmain Edt Spra...   
3             3.3                                                 []   
4             3.4                                                 []   

                                         description  price  \
0                                                 []   None   
1  [Description, The

Extracting duplicate data and labeling of the user_id value to be used for prediction

In [85]:
id = review_json['user_id']
print("The total number of reviews is", id.size)
id_count = (id.value_counts())
print("The user who wrote the review is", id_count.size)
dup_ind = make_list(id_count)

The total number of reviews is 70883
The user who wrote the review is 68934


<ipython-input-79-17c3463facae>:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if series1[i]>1:


In [123]:
new_user_id = id.to_list()
result_dup_id = remove_special(dup_ind)
result_user_id = remove_special(new_user_id)
label_dic = {}
label_df = labeling_user_id(result_user_id,result_dup_id,label_dic)

In [214]:
new_label_df = make_label_list(metadata)

In [215]:
metadata_final = pd.concat([metadata,new_label_df],axis=1)
reviewdata_final = pd.concat([review,label_df],axis = 1)
print(metadata_final)
print(reviewdata_final)

      main_category                                              title  \
0        All Beauty  Howard LC0008 Leather Conditioner, 8-Ounce (4-...   
1        All Beauty  Lurrose 100Pcs Full Cover Fake Toenails Artifi...   
2        All Beauty                         Gold extatic Musk EDT 90ml   
3        All Beauty  Brand New Headrang Face line Contour V-line Ma...   
4        All Beauty  7 Packs Deep Wave Crochet Hair 22 Inch Deep wa...   
...             ...                                                ...   
11252    All Beauty  Yisimei Senegalese Twist Hair 24 Inches 6 Pack...   
11253    All Beauty  3 Packs Crochet Brazilian Braids Hair 20 Inch ...   
11254    All Beauty  Creamily 3 pack Synthetic Braiding Hair Braids...   
11255    All Beauty  Mascara And Liquid Eyeliner Set,2-in-1 Big Eye...   
11256    All Beauty  Black Charcoal Peel off Mask Blackhead Remover...   

       average_rating                                           features  \
0                 4.8              

Data Visualization

In [ ]:
asin = reviewdata_final['asin']
asin_parent = reviewdata_final['parent_asin']
#plt.plot(asin,asin_parent, 'b+')
plt.bar(asin, asin_parent)
plt.show()

Modeling

In [175]:
def text_preprocessing(column):
    #make all words with lower letters
    column = column.str.lower()
    #getting rid of any punctution
    column = column.str.replace('[-=+,#?^@*\"※~ㆍ!』‘|\(\)\[\]\{\}`\'…》\"\"\“\’·]', '', case=False)
    #spliting each sentence to words to apply previous funtions on them
    word_tokens = column.str.split()
    keywords = word_tokens.apply(lambda x: [item for item in x if item not in stop])
    #assemble words of each sentence again and assign them in new column
    for i in range(len(keywords)):
        keywords[i] = " ".join(keywords[i])
        column = keywords
    return column

In [185]:
details_df = metadata_final['details']
details_str_df = details_df.astype(str)
tp_details_df = text_preprocessing(details_str_df)

In [234]:
CV = CountVectorizer()
converted_metrix = CV.fit_transform(tp_details_df)
cs_result = cosine_similarity(converted_metrix)

Prediction

In [178]:
def found_asin(user_id):
  label = reviewdata_final['labeling_user_id']
  label_df_list = label.to_list()
  for data in label_df_list:
    if data == user_id:
      return reviewdata_final[reviewdata_final['labeling_user_id'] == user_id]['parent_asin'].values[0]
  else:
    print("Wrong User_id, Please try again 1~70883.")

In [190]:
def found_data(asin_id):
  asindata = metadata_final['parent_asin']
  asindata_df_list = asindata.to_list()
  for dt in asindata_df_list:
    if dt == asin_id:
      return metadata_final[metadata_final['parent_asin'] == asin_id]['details'].values[0]

In [228]:
review_parent_asin = found_asin(500)
already_buy_title = metadata_final[metadata_final['parent_asin']==review_parent_asin]['title'].values[0]
already_buy_price = metadata_final[metadata_final['title']==already_buy_title]['price'].values[0]
already_buy_rating = reviewdata_final[reviewdata_final['parent_asin']==review_parent_asin]['rating'].values[0]
print(already_buy_title,already_buy_rating,already_buy_price)
user_inform_data = found_data(review_parent_asin)
exdata = metadata_final[metadata_final['details'].str.contains(user_inform_data)]

3-PACK Kiss imPRESS"VEXED & VICIOUS" Medium Length Press-On Manicure Nails 5 None


In [235]:
labeling_id = exdata['labeling_list'].values[0]
score = list(enumerate(cs_result[labeling_id]))
#now sort the similar movies in descending order
sorted_score = sorted(score, key=lambda x:x[1], reverse= True)
#we will ignore the first score because it will give us a 100% score because it's the same movie
sorted_score = sorted_score[1:]
sorted_score[0:10]

[(5610, np.float64(0.8518518518518524)),
 (8989, np.float64(0.7313103409735261)),
 (6469, np.float64(0.7223151185146154)),
 (4055, np.float64(0.714741689891863)),
 (2093, np.float64(0.692820323027551)),
 (3126, np.float64(0.692820323027551)),
 (6729, np.float64(0.692820323027551)),
 (6297, np.float64(0.6793662204867577)),
 (6859, np.float64(0.6735753140545635)),
 (3654, np.float64(0.6719368409052927))]

Prediction

In [232]:
rank = 1
for item in sorted_score:
    title = metadata_final[metadata_final['labeling_list'] == item[0]]['title'].values[0]
    rating = metadata_final[metadata_final['title']==title]['average_rating'].values[0]
    price = metadata_final[metadata_final['title']==title]['price'].values[0]
    detail = metadata_final[metadata_final['title']==title]['details'].values[0]
  #https://aait.tistory.com/88
    #data = str(metadata[metadata['title']==title]['images'])
    #listset_str = ''.join(data)
    #Remove Special characters to space
    #url = re.sub('[-=+,#?^@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', "",listset_str)
    #print(url)
    #url_data = url.str.contains("hi_res: ")
    #Split the data by space
    #print(url_data)
    print(rank,title,rating,price,detail)
    #image_nparray = np.asarray(bytearray(requests.get(url).content), dtype=np.uint8)
    #image = cv2.imdecode(image_nparray, cv2.IMREAD_COLOR)
    #print(image.shape)
    #cv2.imshow('Image from url', image)
    #cv2.waitKey(0)
    rank = rank+1
    if rank > 5:
        break

1 3-PACK KISS imPRESS ON FIRE Short Length Press-On Manicure Nails 4.4 None {"Size": "Small", "Brand": "Broadway", "Style": "French", "Pattern": "Solid", "Coverage": "Full Well Tips", "Is Discontinued By Manufacturer": "No", "Package Dimensions": "5.5 x 5 x 1 inches; 2.88 Ounces", "Item model number": "Kiss", "UPC": "731509192490"}
2 KISS imPRESS Nails Press-On Manicure Nails (BIPA040W02-Harlem Shake) 4.1 None {"Brand": "ImPRESS", "Style": "French", "Pattern": "Solid", "Coverage": "Full Well Tips", "Item Form": "Gel", "Is Discontinued By Manufacturer": "No", "Package Dimensions": "5.5 x 1.95 x 1.1 inches; 2.57 Ounces", "UPC": "731509725681"}
3 BeYou 3Pack 3Colored Salon Nail 100Tips, Professional Fake Nail Tips For Nail salon Nail Shop (25103/25203/25205) (Puple) 4.3 None {"Size": "Large", "Material": "Acrylic", "Style": "French", "Pattern": "Solid", "Coverage": "Full Well Tips", "Is Discontinued By Manufacturer": "No", "Package Dimensions": "6 x 4 x 2.52 inches; 6.4 Ounces", "UPC": "8

Evaluation